In [1]:
!pip install fastai

In [3]:
from fastai.text import *

In [4]:
path = untar_data(URLs.IMDB_SAMPLE)

In [5]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [6]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)
learn.save_encoder('ft_enc')

epoch,train_loss,valid_loss,accuracy,time
0,4.317752,3.892142,0.289122,00:09


epoch,train_loss,valid_loss,accuracy,time
0,3.937712,3.813966,0.298527,00:12


In [7]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))
learn.save('model')

epoch,train_loss,valid_loss,accuracy,time
0,0.614012,0.611361,0.721393,00:06


epoch,train_loss,valid_loss,accuracy,time
0,0.467292,0.483209,0.751244,00:07


In [9]:
learn.predict("This was not a great movie")

(Category tensor(0), tensor(0), tensor([0.6071, 0.3929]))

In [10]:
learn.predict("This was an amazing movie!")

(Category tensor(1), tensor(1), tensor([0.1635, 0.8365]))

In [11]:
learn.predict("I loved this movie")

(Category tensor(1), tensor(1), tensor([0.1267, 0.8733]))